In [ ]:
import os; os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from glob import glob

import numpy as np
import tensorflow as tf

In [ ]:
feature_hidden_dim = 512
lstm_units = 512
classifier_hidden_dim = 256

bow_dim = 2632
visfeat_dim = 4096+8
T = 15

weight_decay = 5e-4
max_epoches = 10000

pos_weight = 0.5 / 0.05
neg_weight = 0.5 / 0.95

data_filter = './data/training_batches/*/*.npz'

exp_name = 'explanation_classifier'
save_dir = './tfmodel/%s/' % exp_name
log_dir = './tb/%s/' % exp_name

os.makedirs(save_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [ ]:
def explanation_classification_model(bow_batch, visfeat_batch, bbox_score_batch, seq_length_batch,
    feature_hidden_dim, lstm_units, classifier_hidden_dim,
    scope='explanation_classifier', reuse=None):
    
    # concatenate all the features, and
    # use a fully-connected layer to map the features to a new dimension
    all_features = tf.concat([bow_batch, visfeat_batch, bbox_score_batch], axis=-1)
    all_features = tf.reshape(all_features, [-1, all_features.get_shape().as_list()[-1]])
    all_features_mapped = tf.layers.dense(all_features, feature_hidden_dim, activation=tf.nn.relu)
    all_features_mapped = tf.reshape(all_features_mapped, [T, -1, feature_hidden_dim])
    
    # feed the features into a LSTM
    cell = tf.contrib.rnn.BasicLSTMCell(lstm_units)
    _, state = tf.nn.dynamic_rnn(cell, all_features_mapped, sequence_length=seq_length_batch,
                             dtype=tf.float32, time_major=True)
    
    # the final classifier: a two-layer network
    embeddings = state.h
    embeddings_mapped = tf.layers.dense(embeddings, classifier_hidden_dim, activation=tf.nn.relu)
    scores = tf.layers.dense(embeddings_mapped, 1)
    scores = tf.reshape(scores, [-1])
    return scores

In [ ]:
# the model
seq_length_batch = tf.placeholder(tf.int32, [None])

bow_batch = tf.placeholder(tf.float32, [T, None, bow_dim])
visfeat_batch = tf.placeholder(tf.float32, [T, None, visfeat_dim])
bbox_score_batch = tf.placeholder(tf.float32, [T, None, 1])

label_batch = tf.placeholder(tf.float32, [None])

In [ ]:
scores = explanation_classification_model(bow_batch, visfeat_batch,
    bbox_score_batch, seq_length_batch, feature_hidden_dim, lstm_units, classifier_hidden_dim)

In [ ]:
# Classification loss
loss_cls = tf.reduce_mean(
    tf.nn.weighted_cross_entropy_with_logits(targets=label_batch,
                                             logits=scores,
                                             pos_weight=pos_weight/neg_weight)) * neg_weight

# Regularization loss
regularization_vars = [v for v in tf.trainable_variables()
                       if v.op.name.endswith('kernel') or v.op.name.endswith('weights')]
print('variables for regularization:')
for v in regularization_vars:
    print(v.op.name)
loss_reg = weight_decay*tf.add_n([tf.nn.l2_loss(v) for v in regularization_vars])

train_op = tf.train.AdamOptimizer().minimize(loss_cls + loss_reg)

In [ ]:
saver = tf.train.Saver(max_to_keep=None)

In [ ]:
# Tensorboard logging
log_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())
tf.summary.scalar("loss_cls", loss_cls)
log_op = tf.summary.merge_all()

In [ ]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
sess.run(tf.global_variables_initializer())

In [ ]:
# load data list
filelist = glob(data_filter)
np.random.shuffle(filelist)

num_batch = len(filelist)

In [ ]:
# save snapshot
save_path = os.path.join(save_dir, '%08d' % 0)
saver.save(sess, save_path)
print('Model saved to %s' % save_path)

total_pos = 0
total_all = 0
for n_epoch in range(max_epoches):
    for n_batch in range(num_batch):
        n_iter = n_epoch*num_batch + n_batch
        
        batch = np.load(filelist[n_batch])
        
        total_pos += np.sum(batch['label_batch'])
        total_all += batch['label_batch'].size
        print('pos ratio: %f = %d / %d' % (total_pos / total_all, total_pos, total_all))
        
        _, summary = sess.run((train_op, log_op),
                              {seq_length_batch: batch['seq_length_batch'],
                               bow_batch: batch['bow_batch'],
                               visfeat_batch: batch['visfeat_batch'],
                               bbox_score_batch: batch['bbox_score_batch'],
                               label_batch: batch['label_batch']})
        print('epoch = %d, batch = %d / %d' % (n_epoch, n_batch, num_batch))
        log_writer.add_summary(summary, n_iter)

    # save snapshot
    save_path = os.path.join(save_dir, '%08d' % (n_epoch+1))
    saver.save(sess, save_path)
    print('Model saved to %s' % save_path)